In [11]:
from numba import cuda, int32

import time

# Generate a synthetic word dataset
#synthetic_dataset = ["word" * i for i in range(1, 10001)]  # List of words of varying lengths

synthetic_dataset = ('/content/datad4_3eji9tmp')

# CUDA kernel for parallel normalization
@cuda.jit
def normalize_parallel(word_array, max_length, result, lengths):
    idx = cuda.grid(1)
    if idx < word_array.shape[0]:
        for j in range(max_length):
            if j < lengths[idx]:
                result[idx * max_length + j] = word_array[idx * max_length + j]
            else:
                result[idx * max_length + j] = ord(' ')

# Helper function to convert the word list to a flat list of ASCII values
def convert_to_flat_list(word_list):
    max_length = max(len(word) for word in word_list)
    flat_list = []
    lengths = []
    for word in word_list:
        flat_list.extend(ord(char) for char in word)
        lengths.append(len(word))
    return flat_list, max_length, lengths

# Convert word list to flat list of ASCII values
word_array, max_length, lengths = convert_to_flat_list(synthetic_dataset)

# Prepare data for CUDA
word_array_device = cuda.to_device(word_array)
lengths_device = cuda.to_device(lengths)
result_device = cuda.device_array(len(synthetic_dataset) * max_length, dtype=int)

# Define CUDA grid dimensions
threads_per_block = 256
blocks_per_grid = (len(synthetic_dataset) + (threads_per_block - 1)) // threads_per_block

# Measure runtime for parallel normalization
start_time = time.time()
normalize_parallel[blocks_per_grid, threads_per_block](word_array_device, max_length, result_device, lengths_device)
cuda.synchronize()
end_time = time.time()
parallel_time = end_time - start_time

# Convert result back to list of strings
result_host = result_device.copy_to_host()
normalized_parallel = ["".join(chr(result_host[i * max_length + j]) for j in range(max_length)).rstrip() for i in range(len(synthetic_dataset))]

print(f"Parallel Normalization Time: {parallel_time:.5f} seconds")



Parallel Normalization Time: 0.13528 seconds


/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


In [12]:
def normalize_sequential(word_list):
    max_length = max(len(word) for word in word_list)
    normalized_list = [(word + ' ' * (max_length - len(word))) for word in word_list]
    return normalized_list

# Measure runtime for sequential normalization
start_time = time.time()
normalized_seq = normalize_sequential(synthetic_dataset)
end_time = time.time()
sequential_time = end_time - start_time

print(f"Sequential Normalization Time: {sequential_time:.5f} seconds")


Sequential Normalization Time: 0.00014 seconds
